#Bài tập


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Dùng bert-base-cased để chạy lại model với input length là 256 trên và so sánh kết quả
2. Thay đổi các lớp FC cuối cùng theo thứ tự sau 100, 50, 20, 10, có thể dùng các activation functions cần thiết.

In [4]:
!pip install -q transformers pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 51.7 MB/s eta 0:00:00


In [5]:
!gdown 1e7iP_EWYV63rOHd-SF-5I1RF7IT2g84t

Downloading...
From: https://drive.google.com/uc?id=1e7iP_EWYV63rOHd-SF-5I1RF7IT2g84t
To: /content/data.csv
100% 3.98M/3.98M [00:00<00:00, 144MB/s]


In [6]:
import torch
import pytorch_lightning as pl
import pandas as pd
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch

In [7]:
# Import data from a CSV file with '\n' as the line terminator
data = pd.read_csv("/content/data.csv", lineterminator='\n')

# Display the first few rows of the dataset for inspection
print(data.head())

# Extract the 'normalized_content' column as the input texts
texts = data.normalized_content
texts = texts.to_list()  # Convert the text data to a list format

# Extract the 'score' column as the labels and normalize them by dividing by 10
labels = data.score.to_list()
labels = [float(i) / 10.0 for i in labels]  # Normalize scores to a range of 0.0 to 1.0

# Output the total number of rows in the dataset
len(data)


                                  normalized_content  score
0                 Very friendly staff. Nice welcome.    8.0
1  staff were very helpful in booking the train o...    8.0
2  It was a superior experience. Accomodation was...   10.0
3  Very helpful owner, any minor issues in the ro...   10.0
4  The staff! They were amazing and so friendly, ...   10.0


14845

In [8]:
class CustomDataset(Dataset):
  def __init__(self,texts,labels,tokenizer,max_length):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_length = max_length
  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text = str(self.texts[idx])
    label = self.labels[idx]
    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = self.max_length,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt'
    )
    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()

    assert input_ids.shape[0] == attention_mask.shape[0] == self.max_length

    return {
        'input_ids' : input_ids,
        'attention_mask' : attention_mask,
        'label' : torch.tensor(label, dtype = torch.float)
    }


In [9]:
import torchmetrics
model_name = 'google-bert/bert-base-cased'
class BERTClassifier(pl.LightningModule):
  def __init__(self,learning_rate = 2e-5):
    super(BERTClassifier,self).__init__()
    self.bert = AutoModel.from_pretrained(model_name)

    self.linear1 = torch.nn.Linear(self.bert.config.hidden_size,100)
    self.linear2 = torch.nn.Linear(100,50)
    self.linear3 = torch.nn.Linear(50,20)
    self.linear4 = torch.nn.Linear(20,10)
    self.linear5 = torch.nn.Linear(10,1)
    self.learning_rate = learning_rate
    self.train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
    self.val_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(input_ids, attention_mask=attention_mask)
        # Feed the [CLS] token representation through linear layers
    linear1 = self.linear1(outputs.last_hidden_state[:, 0, :])
    linear2 = self.linear2(linear1)
    linear3 = self.linear3(linear2)
    linear4 = self.linear4(linear3)
    linear5 =self.linear5(linear4)
    return linear5
  def training_step(self, batch, batch_idx):

    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = torch.reshape(batch['label'], (-1, 1))

        # Forward pass and compute loss
    predicted_value = self(input_ids, attention_mask)
    loss = F.mse_loss(predicted_value, labels)  # Mean squared error loss
    self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)  # Log loss
    return loss

  def validation_step(self, batch, batch_idx):

    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = torch.reshape(batch['label'], (-1, 1))

        # Forward pass and compute loss
    predicted_value = self(input_ids, attention_mask)
    val_loss = F.mse_loss(predicted_value, labels)  # Mean squared error loss
    self.log("val_loss", val_loss, on_step=False, on_epoch=True, prog_bar=True)  # Log loss
    return val_loss
  def test_step(self, batch, batch_idx):

    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = torch.reshape(batch['label'], (-1, 1))

        # Forward pass and compute loss
    predicted_value = self(input_ids, attention_mask)
    test_loss = F.mse_loss(predicted_value, labels)  # Mean squared error loss
    self.log("test_loss", test_loss, on_step=False, on_epoch=True, prog_bar=True)  # Log loss
    return test_loss

  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)  # AdamW optimizer
    return optimizer

In [10]:
class BERTDataModule(pl.LightningDataModule):
    """
    PyTorch Lightning DataModule for handling text data and tokenization.
    """
    def __init__(self, texts, labels, batch_size, max_length, num_workers):
        """
        Initializes data module with text and label datasets.

        Args:
            texts (list): List of input text data.
            labels (list): Corresponding labels for the text data.
            batch_size (int): Batch size for data loading.
            max_length (int): Maximum sequence length for tokenization.
            num_workers (int): Number of workers for data loading.
        """
        super().__init__()
        self.texts = texts
        self.labels = labels
        self.batch_size = batch_size
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)  # Pre-trained tokenizer
        self.num_workers = num_workers

    def setup(self, stage=None):
        """
        Splits the dataset into training, validation, and testing datasets.
        """
        # Split into training and validation datasets
        texts_train, texts_val, labels_train, labels_val = train_test_split(
            self.texts, self.labels, test_size=0.2, random_state=0
        )
        texts_test, _, labels_test, _ = train_test_split(
            texts_val, labels_val, test_size=0.25, random_state=0
        )

        # Create custom datasets
        self.train_dataset = CustomDataset(texts_train, labels_train, self.tokenizer, self.max_length)
        self.val_dataset = CustomDataset(texts_val, labels_val, self.tokenizer, self.max_length)
        self.test_dataset = CustomDataset(texts_test, labels_test, self.tokenizer, self.max_length)

    def train_dataloader(self):
        """
        Returns DataLoader for the training dataset.
        """
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers
        )

    def val_dataloader(self):
        """
        Returns DataLoader for the validation dataset.
        """
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers
        )

    def test_dataloader(self):
        """
        Returns DataLoader for the testing dataset.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers
        )


In [11]:
# Thiết lập các tham số cho mô-đun dữ liệu và mô hình
batch_size = 48  # Số lượng mẫu trên mỗi batch
max_length = 256  # Chiều dài tối đa của mỗi chuỗi đầu vào
num_workers = 1  # Số luồng làm việc để tải dữ liệu

# Khởi tạo BERTDataModule với các tham số đã thiết lập
data_module = BERTDataModule(
    texts=texts,  # Danh sách các chuỗi văn bản
    labels=labels,  # Danh sách nhãn tương ứng
    batch_size=batch_size,  # Kích thước batch
    max_length=max_length,  # Chiều dài tối đa của chuỗi
    num_workers=num_workers  # Số luồng làm việc
)

# Khởi tạo mô hình BERTClassifier
model = BERTClassifier()  # Sử dụng cấu hình mặc định của mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [12]:
data_module.setup()
train_dataloader = data_module.train_dataloader()

In [13]:
next(iter(train_dataloader))['input_ids'][0]

tensor([  101,  1128,  2653,  1111,  1184,  1128,  1243,  1917,  1108, 15354,
         1182,  1198,  1444,   170,  3205,  1106,  2215,  1229,  3074,  1111,
         1103,  2147,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [14]:
# Cài đặt callback ModelCheckpoint để lưu mô hình tốt nhất
model_checkpoint = ModelCheckpoint(
    dirpath='/content/drive/MyDrive/GIt/checkpoint/',  # Thư mục lưu trữ mô hình
    monitor="val_loss",  # Theo dõi giá trị tổn thất trên tập validation
    verbose=True,  # Hiển thị thông tin khi lưu checkpoint
    mode="min",  # Chọn mô hình có giá trị tổn thất nhỏ nhất
    save_top_k=1  # Lưu chỉ một mô hình tốt nhất
)

# Cài đặt callback EarlyStopping để dừng sớm nếu không có cải thiện
early_stopping = EarlyStopping(
    monitor="val_loss",  # Theo dõi giá trị tổn thất trên tập validation
    mode="min",  # Dừng khi giá trị tổn thất không còn giảm
    min_delta=1e-4,  # Ngưỡng tối thiểu để xác định cải thiện
    patience=5  # Số epoch chờ trước khi dừng
)

# Tập hợp các callback
callbacks = [model_checkpoint, early_stopping]

# Cài đặt Trainer của PyTorch Lightning
trainer = pl.Trainer(
    max_epochs=10,  # Số epoch tối đa để huấn luyện
    detect_anomaly=True,  # Bật phát hiện bất thường trong quá trình huấn luyện
    callbacks=callbacks  # Gắn các callback đã thiết lập
)

# Huấn luyện mô hình với dữ liệu
trainer.fit(model, data_module)


INFO:pytorch_lightning.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | bert      | BertModel          | 108 M  | eval 
1 | linear1   | Linear             | 76.9 K | train
2 | linear2   | Linear             | 5.0 K  | train
3 | linear3   | Linear             | 1.0 K  | train
4 | linear4   | Linear             | 210    | train
5 | linear5   | Linear             | 11     | train
6 | train_acc |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 248: 'val_loss' reached 0.01175 (best 0.01175), saving model to '/content/drive/MyDrive/GIt/checkpoint/epoch=0-step=248.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 1984: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 2232: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 2480: 'val_loss' reached 0.01030 (best 0.01030), saving model to '/content/drive/MyDrive/GIt/checkpoint/epoch=9-step=2480.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [15]:
final_metrics = trainer.callback_metrics
print(f"Final Metrics: {final_metrics}")

Final Metrics: {'val_loss': tensor(0.0103), 'train_loss': tensor(0.0005)}
